# FinTech Project Three
### Summary

### Section 1. Data Preparation
Import and initialization of libraries 

In [1]:
# Imports of OS and API libraries

import os
from dotenv import load_dotenv
import requests
from binance import Client

In [2]:
# Imports of Pandas and related packages
import pandas as pd
import numpy as np
%matplotlib inline

In [3]:
# Setting up Binance API keys and client for data endpoints
load_dotenv()
BINANCE_API_KEY=os.getenv('BINANCE_API_KEY')
BINANCE_SECRET_KEY=os.getenv('BINANCE_SECRET_KEY')
client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY, tld='us')

In [6]:
# Fetching Bitcoin USD data from June 2017 to June 2022 for the interval of 1-Day
bitcoin_data = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY, "01 Jun, 2017", "30 Jun, 2022")

In [7]:
# Converting into a Dataframe
bitcoin_df = pd.DataFrame(bitcoin_data)

# Renaming columns to their correct labels
bitcoin_df.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'NaN']

# Converting open_time and close_time data values to date-time format
bitcoin_df['open_time'] = pd.to_datetime(bitcoin_df['open_time'], unit='ms')
bitcoin_df['close_time'] = pd.to_datetime(bitcoin_df['close_time'], unit='ms')

# Dropping last column NaN
bitcoin_df.drop(columns='NaN', inplace=True)

# Display Dataframe
bitcoin_df

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
0,2019-09-23,9930.13000000,9930.13000000,9930.13000000,9930.13000000,0.00100000,2019-09-23 23:59:59.999,9.93013000,1,0.00000000,0.00000000
1,2019-09-24,9637.93000000,9665.05000000,7996.45000000,8519.67000000,202.35079900,2019-09-24 23:59:59.999,1739432.08920509,2345,125.93546400,1075623.60280331
2,2019-09-25,8528.12000000,8724.77000000,8247.78000000,8435.28000000,55.41797100,2019-09-25 23:59:59.999,469361.69055629,763,29.05942700,246049.20514584
3,2019-09-26,8434.45000000,8453.20000000,7761.87000000,8064.41000000,41.88007900,2019-09-26 23:59:59.999,340923.66601370,1023,19.80884900,161115.44149782
4,2019-09-27,8078.75000000,8259.96000000,7858.58000000,8178.73000000,47.14290400,2019-09-27 23:59:59.999,379917.99846542,990,23.90374200,192755.96772991
...,...,...,...,...,...,...,...,...,...,...,...
1007,2022-06-26,21489.82000000,21882.15000000,20979.11000000,21047.27000000,586.39114300,2022-06-26 23:59:59.999,12536451.28730396,17433,282.99055700,6054924.37196783
1008,2022-06-27,21035.02000000,21538.40000000,20520.44000000,20729.39000000,706.57827600,2022-06-27 23:59:59.999,14829893.61772424,21305,363.59247400,7630948.56107983
1009,2022-06-28,20735.95000000,21210.85000000,20197.04000000,20280.68000000,1245.63697000,2022-06-28 23:59:59.999,25737921.90414807,25714,616.99736900,12747493.58986636
1010,2022-06-29,20275.20000000,20439.60000000,19851.47000000,20117.99000000,2589.68285300,2022-06-29 23:59:59.999,52120920.38769744,47615,1277.56600800,25712880.05231522


In [9]:
# Fixing datatype to replace closing price, open price, high, low and volume from an object to a float
bitcoin_df = bitcoin_df.assign(
  close = lambda df: df['close'].str.replace('',''),
  open = lambda df: df['open'].str.replace('',''),
  high = lambda df: df['high'].str.replace('',''),
  low = lambda df: df['low'].str.replace('',''),
  volume = lambda df: df['volume'].str.replace('',''),
  quote_asset_volume = lambda df: df['quote_asset_volume'].str.replace('',''),
  taker_buy_base_asset_volume = lambda df: df['taker_buy_base_asset_volume'].str.replace('',''),
  taker_buy_quote_asset_volume = lambda df: df['taker_buy_quote_asset_volume'].str.replace('','')
)

In [10]:
bitcoin_df = bitcoin_df.assign(
  close = lambda df: df['close'].astype('float'),
  open = lambda df: df['open'].astype('float'),
  high = lambda df: df['high'].astype('float'),
  low = lambda df: df['low'].astype('float'),
  volume = lambda df: df['volume'].astype('float'),
  quote_asset_volume = lambda df: df['quote_asset_volume'].astype('float'),
  taker_buy_base_asset_volume = lambda df: df['taker_buy_base_asset_volume'].astype('float'),
  taker_buy_quote_asset_volume = lambda df: df['taker_buy_quote_asset_volume'].astype('float'),
  number_of_trades = lambda df: df['number_of_trades'].astype('float')
)

In [11]:
# Pickle dataframe
bitcoin_df.to_pickle('bitcoin_df.pkl')

### Section 2. Trading Scenario and Strategy Development
#### Scenario - Archetype of trader
#### Strategy - Explanation of trading strategy and technical indicators utilized to evaluate the strategy

In [ ]:
# Read in the pickled dataframe
bitcoin_df = pd.read_pickle('bitcoin_df.pkl')

In [ ]:
## Create dataframes with momentum technical indicators

In [ ]:
## Create dataframe with trend technical indicators

In [ ]:
## Create dataframe with volume technical indicators

In [ ]:
## Develop ways to create signals

### Section 3. Feature evaluation and feature engineering with RandomForest

### Section 4. Development of preliminary crypto-trading strategy by identifying best features/indicators

### Section 5. Data visualization and dashboarding

### Section 6. Deep learning with TensorFlow for Predictive Modeling

### Section 7. Backtesting and model evaluation